# seqFISH Mouse Organogenesis Imputed Extended

- **Creator**: Sebastian Birk (<sebastian.birk@helmholtz-munich.de>)
- **Date of Creation:** 01.08.2024
- **Date of Last Modification:** 03.08.2024

## 1. Setup

Run this notebook in the nichecompass-reproducibility environment, installable from ```('../../../envs/environment.yaml')```.

### 1.1 Import Libraries

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../../utils")

In [ ]:
import os
import warnings

import anndata as ad
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import scipy.sparse as sp
from sklearn.preprocessing import MinMaxScaler
import squidpy as sq
from nichecompass.models import NicheCompass
from nichecompass.utils import create_new_color_dict

from analysis_utils import plot_category_in_latent_and_physical_space

### 1.2 Define Parameters

In [ ]:
dataset = "seqfish_mouse_organogenesis_imputed"
species = "mouse"
counts_key = "counts"
cat_covariates_keys = "batch"
gp_names_key = "nichecompass_gp_names"
differential_gp_test_results_key = "nichecompass_differential_gp_test_results"
spatial_key = "spatial"
adj_key = "spatial_connectivities"
n_neighbors = 8
latent_key = "nichecompass_latent"
sample_key = "sample"
spot_size = 0.05 

load_timestamps = ["01082024_191836_3", # embryo 1,2 (reference) and embryo 3 (query)
                   "01082024_191835_2", # embryo 1,3 (reference and embryo 2 (query)
                   "02082024_111323_6", # all embryos (reference) and random seed 42
                   "02082024_111324_6", # all embryos (reference) and random seed 43
                   "02082024_002349_5", # all embryos (reference) and KNN 12
                   "02082024_002350_4", # all embryos (reference) and KNN 4
                  ]
batches = ["batch1", "batch2", "batch3", "batch4", "batch5", "batch6"]

### 1.3 Run Notebook Setup

In [ ]:
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### 1.4 Configure Paths and Create Directories

In [ ]:
# Define paths
ga_data_folder_path = "../../datasets/ga_data"
gp_data_folder_path = "../../datasets/gp_data"
st_data_folder_path = "../../datasets/st_data"
st_data_gold_folder_path = st_data_folder_path + "/gold"
omnipath_lr_network_file_path = f"{gp_data_folder_path}/omnipath_lr_network.csv"
nichenet_lr_network_file_path = f"{gp_data_folder_path}/nichenet_lr_network_v2_{species}.csv"
nichenet_ligand_target_matrix_file_path = f"{gp_data_folder_path}/nichenet_ligand_target_matrix_v2_{species}.csv"
mebocost_enzyme_sensor_interactions_folder_path = f"{gp_data_folder_path}/metabolite_enzyme_sensor_gps"
gene_orthologs_mapping_file_path = f"{ga_data_folder_path}/human_mouse_gene_orthologs.csv"
artifacts_folder_path = f"../../artifacts"

## 2. Analysis

### 2.1. Leave Out Embryo 3

In [ ]:
model_label = "reference_query"
model_folder_path = f"{artifacts_folder_path}/{dataset}/models/{model_label}/{load_timestamps[0]}"
figure_folder_path = f"{artifacts_folder_path}/{dataset}/figures/{model_label}/{load_timestamps[0]}"


# Create required directories
os.makedirs(figure_folder_path, exist_ok=True)

model = NicheCompass.load(dir_path=model_folder_path,
                          adata=None,
                          adata_file_name=f"{dataset}_{model_label}.h5ad",
                          gp_names_key=gp_names_key)

In [ ]:
samples = model.adata.obs[sample_key].unique().tolist()

In [ ]:
# Identify and plot niches
latent_leiden_resolution = 0.25
latent_cluster_key = f"latent_leiden_{str(latent_leiden_resolution)}"

sc.tl.leiden(adata=model.adata,
             resolution=latent_leiden_resolution,
             key_added=latent_cluster_key,
             neighbors_key=latent_key)

latent_cluster_colors = create_new_color_dict(
    adata=model.adata,
    color_palette="default",
    cat_key=latent_cluster_key)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "latent_clusters_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=None,
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot CNS niches
forebrain_cluster = "3"
midbrain_cluster = "4"
hindbrain_cluster = "11"
floorplate_cluster = "19"

latent_cluster_colors[forebrain_cluster] = "#B497E7"
latent_cluster_colors[midbrain_cluster] = "#FF69B4"
latent_cluster_colors[hindbrain_cluster] = "#276A8C"
latent_cluster_colors[floorplate_cluster] = "#C38D9E"

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "forebrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[forebrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "midbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[midbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "hindbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[hindbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "floorplate_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[floorplate_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot gut niches
ventralgut_cluster = "10"
dorsalgut_cluster = "20"

latent_cluster_colors[ventralgut_cluster] = "#F6CF71"
latent_cluster_colors[dorsalgut_cluster] = "#9D88A2"

# Ventral Gut
save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "ventralgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[ventralgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "dorsalgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[dorsalgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Run differential GP testing
selected_cats = None
log_bayes_factor_thresh = 2.3

enriched_gps = model.run_differential_gp_tests(
    cat_key=latent_cluster_key,
    selected_cats=selected_cats,
    comparison_cats="rest",
    log_bayes_factor_thresh=log_bayes_factor_thresh)

# Check whether GPs are enriched
niche_gps = {}
niche_gps[forebrain_cluster] = ["PCSK1N_ligand_receptor_GP", "DKK1_ligand_receptor_GP"]
niche_gps[midbrain_cluster] = ["Fgf17_ligand_receptor_target_gene_GP", "Efna2_ligand_receptor_target_gene_GP"]
niche_gps[hindbrain_cluster] = ["Gdf10_ligand_receptor_target_gene_GP", "Fgf3_ligand_receptor_target_gene_GP"]
niche_gps[floorplate_cluster] = ["CALCA_ligand_receptor_GP", "Shh_ligand_receptor_target_gene_GP"]
niche_gps[ventralgut_cluster] = ["IHH_ligand_receptor_GP", "Spint1_ligand_receptor_target_gene_GP"]
niche_gps[dorsalgut_cluster] = ["Cthrc1_ligand_receptor_target_gene_GP", "PDGFC_ligand_receptor_GP"]

overlap_gps = {}
for niche, gps in niche_gps.items():
    niche_enriched_gps = model.adata.uns[differential_gp_test_results_key][
        model.adata.uns[differential_gp_test_results_key]["category"] == niche]["gene_program"].values.tolist()
    overlap_gps[niche] = [gp for gp in gps if gp in niche_enriched_gps]

# Correct sign of GPs
gp_summary_df = model.get_gp_summary()
model.add_active_gp_scores_to_obs()

for idx, row in gp_summary_df[gp_summary_df["gp_active"]].iterrows():
    if np.array(row["gp_source_genes_weights"]).sum() < 0:
        model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
    elif np.array(row["gp_source_genes_weights"]).sum() == 0:
        if np.array(row["gp_target_genes_weights"]).sum() < 0:
            model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
            
all_gps = []
for sublist in list(niche_gps.values()):
    all_gps.extend(sublist)

In [ ]:
# Plot CNS GPs
adata_top = model.adata[model.adata.obsm["spatial"][:,1] < 0]
adata_top = adata_top[adata_top.obs["sample"] == "embryo2"]

for gp in all_gps:
    if gp in adata_top.obs.columns:
        scaler = MinMaxScaler()
        # Fit and transform the column to normalize its values
        adata_top.obs[f'{gp}_normalized'] = scaler.fit_transform(adata_top.obs[[gp]])

color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=4,
                        ncols=2,
                        figsize=(15, 15))

# Forebrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[forebrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Pcsk1n GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[forebrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Dkk1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

# Midbrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[midbrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Fgf17 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[midbrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Efna2 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)

# Floor plate
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[floorplate_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Calca GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[floorplate_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Shh GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 1],
    show=False)

# Hindbrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[hindbrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Gdf10 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[3, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[hindbrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Fgf3 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[3, 1],
    show=False)

plt.show()
fig.savefig(f"{figure_folder_path}/brain_enriched_gps_top.svg",
            bbox_inches="tight")

In [ ]:
# Plot gut GPs
adata_gut = model.adata[model.adata.obsm["spatial"][:,1] < 2.8]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,1] > -1.6]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,0] < 1.3]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,0] > -1.2]
adata_gut = adata_gut[adata_gut.obs["sample"] == "embryo2"]

for gp in all_gps:
    if gp in adata_gut.obs.columns:
        scaler = MinMaxScaler()
        # Fit and transform the column to normalize its values
        adata_gut.obs[f'{gp}_normalized'] = scaler.fit_transform(adata_gut.obs[[gp]])

color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=2,
                        ncols=2,
                        figsize=(15, 15))

# Ventral Gut
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[dorsalgut_cluster][0]}", # not normalized (to match original manuscript)
    color_map=color_map,
    spot_size=spot_size,
    title=f"Cthrc1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[dorsalgut_cluster][1]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Pdgfc GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

# Dorsal Gut
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[ventralgut_cluster][0]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Ihh GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[ventralgut_cluster][1]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Spint1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)
fig.savefig(f"{figure_folder_path}/gut_enriched_gps.svg",
            bbox_inches="tight")

### 2.2. Leave Out Embryo 2

In [ ]:
model_label = "reference_query"
model_folder_path = f"{artifacts_folder_path}/{dataset}/models/{model_label}/{load_timestamps[1]}"
figure_folder_path = f"{artifacts_folder_path}/{dataset}/figures/{model_label}/{load_timestamps[1]}"


# Create required directories
os.makedirs(figure_folder_path, exist_ok=True)

model = NicheCompass.load(dir_path=model_folder_path,
                          adata=None,
                          adata_file_name=f"{dataset}_{model_label}.h5ad",
                          gp_names_key=gp_names_key)

In [ ]:
samples = model.adata.obs[sample_key].unique().tolist()

In [ ]:
# Identify and plot niches
latent_leiden_resolution = 0.3
latent_cluster_key = f"latent_leiden_{str(latent_leiden_resolution)}"

sc.tl.leiden(adata=model.adata,
             resolution=latent_leiden_resolution,
             key_added=latent_cluster_key,
             neighbors_key=latent_key)

latent_cluster_colors = create_new_color_dict(
    adata=model.adata,
    color_palette="default",
    cat_key=latent_cluster_key)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "latent_clusters_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=None,
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot CNS niches
forebrain_cluster = "3"
midbrain_cluster = "2"
hindbrain_cluster = "7"
floorplate_cluster = "20"

latent_cluster_colors[forebrain_cluster] = "#B497E7"
latent_cluster_colors[midbrain_cluster] = "#FF69B4"
latent_cluster_colors[hindbrain_cluster] = "#276A8C"
latent_cluster_colors[floorplate_cluster] = "#C38D9E"

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "forebrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[forebrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "midbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[midbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "hindbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[hindbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "floorplate_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[floorplate_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot gut niches
ventralgut_cluster = "1"
dorsalgut_cluster = "23"

latent_cluster_colors[ventralgut_cluster] = "#F6CF71"
latent_cluster_colors[dorsalgut_cluster] = "#9D88A2"

# Ventral Gut
save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "ventralgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[ventralgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "dorsalgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[dorsalgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Run differential GP testing
selected_cats = None
log_bayes_factor_thresh = 2.3

enriched_gps = model.run_differential_gp_tests(
    cat_key=latent_cluster_key,
    selected_cats=selected_cats,
    comparison_cats="rest",
    log_bayes_factor_thresh=log_bayes_factor_thresh)

# Check whether GPs are enriched
niche_gps = {}
niche_gps[forebrain_cluster] = ["PCSK1N_ligand_receptor_GP", "DKK1_ligand_receptor_GP"]
niche_gps[midbrain_cluster] = ["Fgf17_ligand_receptor_target_gene_GP", "Efna2_ligand_receptor_target_gene_GP"]
niche_gps[hindbrain_cluster] = ["Gdf10_ligand_receptor_target_gene_GP", "Fgf3_ligand_receptor_target_gene_GP"]
niche_gps[floorplate_cluster] = ["CALCA_ligand_receptor_GP", "Shh_ligand_receptor_target_gene_GP"]
niche_gps[ventralgut_cluster] = ["IHH_ligand_receptor_GP", "Spint1_ligand_receptor_target_gene_GP"]
niche_gps[dorsalgut_cluster] = ["Cthrc1_ligand_receptor_target_gene_GP", "PDGFC_ligand_receptor_GP"]

overlap_gps = {}
for niche, gps in niche_gps.items():
    niche_enriched_gps = model.adata.uns[differential_gp_test_results_key][
        model.adata.uns[differential_gp_test_results_key]["category"] == niche]["gene_program"].values.tolist()
    overlap_gps[niche] = [gp for gp in gps if gp in niche_enriched_gps]

# Correct sign of GPs
gp_summary_df = model.get_gp_summary()
model.add_active_gp_scores_to_obs()

for idx, row in gp_summary_df[gp_summary_df["gp_active"]].iterrows():
    if np.array(row["gp_source_genes_weights"]).sum() < 0:
        model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
    elif np.array(row["gp_source_genes_weights"]).sum() == 0:
        if np.array(row["gp_target_genes_weights"]).sum() < 0:
            model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
            
all_gps = []
for sublist in list(niche_gps.values()):
    all_gps.extend(sublist)

In [ ]:
# Plot CNS GPs
adata_top = model.adata[model.adata.obsm["spatial"][:,1] < 0]
adata_top = adata_top[adata_top.obs["sample"] == "embryo2"]

for gp in all_gps:
    if gp in adata_top.obs.columns:
        scaler = MinMaxScaler()
        # Fit and transform the column to normalize its values
        adata_top.obs[f'{gp}_normalized'] = scaler.fit_transform(adata_top.obs[[gp]])

color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=4,
                        ncols=2,
                        figsize=(15, 15))

# Forebrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[forebrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Pcsk1n GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[forebrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Dkk1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

# Midbrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[midbrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Fgf17 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[midbrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Efna2 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)

# Floor plate
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[floorplate_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Calca GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[floorplate_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Shh GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 1],
    show=False)

# Hindbrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[hindbrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Gdf10 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[3, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[hindbrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Fgf3 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[3, 1],
    show=False)

plt.show()
fig.savefig(f"{figure_folder_path}/brain_enriched_gps_top.svg",
            bbox_inches="tight")

In [ ]:
# Plot gut GPs
adata_gut = model.adata[model.adata.obsm["spatial"][:,1] < 2.8]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,1] > -1.6]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,0] < 1.3]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,0] > -1.2]
adata_gut = adata_gut[adata_gut.obs["sample"] == "embryo2"]

for gp in all_gps:
    if gp in adata_gut.obs.columns:
        scaler = MinMaxScaler()
        # Fit and transform the column to normalize its values
        adata_gut.obs[f'{gp}_normalized'] = scaler.fit_transform(adata_gut.obs[[gp]])

color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=2,
                        ncols=2,
                        figsize=(15, 15))

# Ventral Gut
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[dorsalgut_cluster][0]}", # not normalized (to match original manuscript)
    color_map=color_map,
    spot_size=spot_size,
    title=f"Cthrc1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[dorsalgut_cluster][1]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Pdgfc GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

# Dorsal Gut
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[ventralgut_cluster][0]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Ihh GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[ventralgut_cluster][1]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Spint1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/gut_enriched_gps.svg",
            bbox_inches="tight")

### 2.3. All Emybros Random Seed 42

In [ ]:
model_label = "reference"
model_folder_path = f"{artifacts_folder_path}/{dataset}/models/{model_label}/{load_timestamps[2]}"
figure_folder_path = f"{artifacts_folder_path}/{dataset}/figures/{model_label}/{load_timestamps[2]}"


# Create required directories
os.makedirs(figure_folder_path, exist_ok=True)

model = NicheCompass.load(dir_path=model_folder_path,
                          adata=None,
                          adata_file_name=f"{dataset}_{model_label}.h5ad",
                          gp_names_key=gp_names_key)

In [ ]:
samples = model.adata.obs[sample_key].unique().tolist()

In [ ]:
# Identify and plot niches
latent_leiden_resolution = 0.32
latent_cluster_key = f"latent_leiden_{str(latent_leiden_resolution)}"

sc.tl.leiden(adata=model.adata,
             resolution=latent_leiden_resolution,
             key_added=latent_cluster_key,
             neighbors_key=latent_key)

latent_cluster_colors = create_new_color_dict(
    adata=model.adata,
    color_palette="default",
    cat_key=latent_cluster_key)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "latent_clusters_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=None,
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot CNS niches
forebrain_cluster = "4"
midbrain_cluster = "6"
hindbrain_cluster = "3"
floorplate_cluster = "16"

latent_cluster_colors[forebrain_cluster] = "#B497E7"
latent_cluster_colors[midbrain_cluster] = "#FF69B4"
latent_cluster_colors[hindbrain_cluster] = "#276A8C"
latent_cluster_colors[floorplate_cluster] = "#C38D9E"

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "forebrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[forebrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "midbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[midbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "hindbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[hindbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "floorplate_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[floorplate_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot gut niches
ventralgut_cluster = "5"
dorsalgut_cluster = "19"

latent_cluster_colors[ventralgut_cluster] = "#F6CF71"
latent_cluster_colors[dorsalgut_cluster] = "#9D88A2"

# Ventral Gut
save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "ventralgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[ventralgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "dorsalgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[dorsalgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Run differential GP testing
selected_cats = None
log_bayes_factor_thresh = 2.3

enriched_gps = model.run_differential_gp_tests(
    cat_key=latent_cluster_key,
    selected_cats=selected_cats,
    comparison_cats="rest",
    log_bayes_factor_thresh=log_bayes_factor_thresh)

# Check whether GPs are enriched
niche_gps = {}
niche_gps[forebrain_cluster] = ["PCSK1N_ligand_receptor_GP", "DKK1_ligand_receptor_GP"]
niche_gps[midbrain_cluster] = ["Fgf17_ligand_receptor_target_gene_GP", "Efna2_ligand_receptor_target_gene_GP"]
niche_gps[hindbrain_cluster] = ["Gdf10_ligand_receptor_target_gene_GP", "Fgf3_ligand_receptor_target_gene_GP"]
niche_gps[floorplate_cluster] = ["CALCA_ligand_receptor_GP", "Shh_ligand_receptor_target_gene_GP"]
niche_gps[ventralgut_cluster] = ["IHH_ligand_receptor_GP", "Spint1_ligand_receptor_target_gene_GP"]
niche_gps[dorsalgut_cluster] = ["Cthrc1_ligand_receptor_target_gene_GP", "PDGFC_ligand_receptor_GP"]

overlap_gps = {}
for niche, gps in niche_gps.items():
    niche_enriched_gps = model.adata.uns[differential_gp_test_results_key][
        model.adata.uns[differential_gp_test_results_key]["category"] == niche]["gene_program"].values.tolist()
    overlap_gps[niche] = [gp for gp in gps if gp in niche_enriched_gps]

# Correct sign of GPs
gp_summary_df = model.get_gp_summary()
model.add_active_gp_scores_to_obs()

for idx, row in gp_summary_df[gp_summary_df["gp_active"]].iterrows():
    if np.array(row["gp_source_genes_weights"]).sum() < 0:
        model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
    elif np.array(row["gp_source_genes_weights"]).sum() == 0:
        if np.array(row["gp_target_genes_weights"]).sum() < 0:
            model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
            
all_gps = []
for sublist in list(niche_gps.values()):
    all_gps.extend(sublist)

In [ ]:
# Plot CNS GPs
adata_top = model.adata[model.adata.obsm["spatial"][:,1] < 0]
adata_top = adata_top[adata_top.obs["sample"] == "embryo2"]

for gp in all_gps:
    if gp in adata_top.obs.columns:
        scaler = MinMaxScaler()
        # Fit and transform the column to normalize its values
        adata_top.obs[f'{gp}_normalized'] = scaler.fit_transform(adata_top.obs[[gp]])

color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=4,
                        ncols=2,
                        figsize=(15, 15))

# Forebrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[forebrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Pcsk1n GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[forebrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Dkk1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

# Midbrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[midbrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Fgf17 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[midbrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Efna2 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)

# Floor plate
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[floorplate_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Calca GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[floorplate_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Shh GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 1],
    show=False)

# Hindbrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[hindbrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Gdf10 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[3, 0],
    show=False)
#sc.pl.spatial(
#    adata=adata_top,
#    color=f"{niche_gps[hindbrain_cluster][1]}_normalized",
#    color_map=color_map,
#    spot_size=spot_size,
#    title=f"Fgf3 GP",
#    legend_loc=None,
#    colorbar_loc="bottom",
#    ax=axs[3, 1],
#    show=False) # inactive GP

plt.show()
fig.savefig(f"{figure_folder_path}/brain_enriched_gps_top.svg",
            bbox_inches="tight")

In [ ]:
# Plot gut GPs
adata_gut = model.adata[model.adata.obsm["spatial"][:,1] < 2.8]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,1] > -1.6]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,0] < 1.3]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,0] > -1.2]
adata_gut = adata_gut[adata_gut.obs["sample"] == "embryo2"]

for gp in all_gps:
    if gp in adata_gut.obs.columns:
        scaler = MinMaxScaler()
        # Fit and transform the column to normalize its values
        adata_gut.obs[f'{gp}_normalized'] = scaler.fit_transform(adata_gut.obs[[gp]])

color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=2,
                        ncols=2,
                        figsize=(15, 15))

# Ventral Gut
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[dorsalgut_cluster][0]}", # not normalized (to match original manuscript)
    color_map=color_map,
    spot_size=spot_size,
    title=f"Cthrc1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[dorsalgut_cluster][1]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Pdgfc GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

# Dorsal Gut
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[ventralgut_cluster][0]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Ihh GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[ventralgut_cluster][1]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Spint1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/gut_enriched_gps.svg",
            bbox_inches="tight")

### 2.4. All Emybros Random Seed 43

In [ ]:
model_label = "reference"
model_folder_path = f"{artifacts_folder_path}/{dataset}/models/{model_label}/{load_timestamps[3]}"
figure_folder_path = f"{artifacts_folder_path}/{dataset}/figures/{model_label}/{load_timestamps[3]}"


# Create required directories
os.makedirs(figure_folder_path, exist_ok=True)

model = NicheCompass.load(dir_path=model_folder_path,
                          adata=None,
                          adata_file_name=f"{dataset}_{model_label}.h5ad",
                          gp_names_key=gp_names_key)

In [ ]:
samples = model.adata.obs[sample_key].unique().tolist()

In [ ]:
# Identify and plot niches
latent_leiden_resolution = 0.32
latent_cluster_key = f"latent_leiden_{str(latent_leiden_resolution)}"

sc.tl.leiden(adata=model.adata,
             resolution=latent_leiden_resolution,
             key_added=latent_cluster_key,
             neighbors_key=latent_key)

latent_cluster_colors = create_new_color_dict(
    adata=model.adata,
    color_palette="default",
    cat_key=latent_cluster_key)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "latent_clusters_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=None,
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot CNS niches
forebrain_cluster = "5"
midbrain_cluster = "4"
hindbrain_cluster = "7"
floorplate_cluster = "17"

latent_cluster_colors[forebrain_cluster] = "#B497E7"
latent_cluster_colors[midbrain_cluster] = "#FF69B4"
latent_cluster_colors[hindbrain_cluster] = "#276A8C"
latent_cluster_colors[floorplate_cluster] = "#C38D9E"

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "forebrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[forebrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "midbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[midbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "hindbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[hindbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "floorplate_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[floorplate_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot gut niches
ventralgut_cluster = "1"
dorsalgut_cluster = "20"

latent_cluster_colors[ventralgut_cluster] = "#F6CF71"
latent_cluster_colors[dorsalgut_cluster] = "#9D88A2"

# Ventral Gut
save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "ventralgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[ventralgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "dorsalgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[dorsalgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Run differential GP testing
selected_cats = None
log_bayes_factor_thresh = 2.3

enriched_gps = model.run_differential_gp_tests(
    cat_key=latent_cluster_key,
    selected_cats=selected_cats,
    comparison_cats="rest",
    log_bayes_factor_thresh=log_bayes_factor_thresh)

# Check whether GPs are enriched
niche_gps = {}
niche_gps[forebrain_cluster] = ["PCSK1N_ligand_receptor_GP", "DKK1_ligand_receptor_GP"]
niche_gps[midbrain_cluster] = ["Fgf17_ligand_receptor_target_gene_GP", "Efna2_ligand_receptor_target_gene_GP"]
niche_gps[hindbrain_cluster] = ["Gdf10_ligand_receptor_target_gene_GP", "Fgf3_ligand_receptor_target_gene_GP"]
niche_gps[floorplate_cluster] = ["CALCA_ligand_receptor_GP", "Shh_ligand_receptor_target_gene_GP"]
niche_gps[ventralgut_cluster] = ["IHH_ligand_receptor_GP", "Spint1_ligand_receptor_target_gene_GP"]
niche_gps[dorsalgut_cluster] = ["Cthrc1_ligand_receptor_target_gene_GP", "PDGFC_ligand_receptor_GP"]

overlap_gps = {}
for niche, gps in niche_gps.items():
    niche_enriched_gps = model.adata.uns[differential_gp_test_results_key][
        model.adata.uns[differential_gp_test_results_key]["category"] == niche]["gene_program"].values.tolist()
    overlap_gps[niche] = [gp for gp in gps if gp in niche_enriched_gps]

# Correct sign of GPs
gp_summary_df = model.get_gp_summary()
model.add_active_gp_scores_to_obs()

for idx, row in gp_summary_df[gp_summary_df["gp_active"]].iterrows():
    if np.array(row["gp_source_genes_weights"]).sum() < 0:
        model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
    elif np.array(row["gp_source_genes_weights"]).sum() == 0:
        if np.array(row["gp_target_genes_weights"]).sum() < 0:
            model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
            
all_gps = []
for sublist in list(niche_gps.values()):
    all_gps.extend(sublist)

In [ ]:
# Plot CNS GPs
adata_top = model.adata[model.adata.obsm["spatial"][:,1] < 0]
adata_top = adata_top[adata_top.obs["sample"] == "embryo2"]

for gp in all_gps:
    if gp in adata_top.obs.columns:
        scaler = MinMaxScaler()
        # Fit and transform the column to normalize its values
        adata_top.obs[f'{gp}_normalized'] = scaler.fit_transform(adata_top.obs[[gp]])

color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=4,
                        ncols=2,
                        figsize=(15, 15))

# Forebrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[forebrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Pcsk1n GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[forebrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Dkk1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

# Midbrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[midbrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Fgf17 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[midbrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Efna2 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)

# Floor plate
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[floorplate_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Calca GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[floorplate_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Shh GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 1],
    show=False)

# Hindbrain
#sc.pl.spatial(
#    adata=adata_top,
#    color=f"{niche_gps[hindbrain_cluster][0]}_normalized",
#    color_map=color_map,
#    spot_size=spot_size,
#    title=f"Gdf10 GP",
#    legend_loc=None,
#    colorbar_loc="bottom",
#    ax=axs[3, 0],
#    show=False) # inactive GP
#sc.pl.spatial(
#    adata=adata_top,
#    color=f"{niche_gps[hindbrain_cluster][1]}_normalized",
#    color_map=color_map,
#    spot_size=spot_size,
#    title=f"Fgf3 GP",
#    legend_loc=None,
#    colorbar_loc="bottom",
#    ax=axs[3, 1],
#    show=False)

plt.show()
fig.savefig(f"{figure_folder_path}/brain_enriched_gps_top.svg",
            bbox_inches="tight")

In [ ]:
# Plot gut GPs
adata_gut = model.adata[model.adata.obsm["spatial"][:,1] < 2.8]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,1] > -1.6]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,0] < 1.3]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,0] > -1.2]
adata_gut = adata_gut[adata_gut.obs["sample"] == "embryo2"]

for gp in all_gps:
    if gp in adata_gut.obs.columns:
        scaler = MinMaxScaler()
        # Fit and transform the column to normalize its values
        adata_gut.obs[f'{gp}_normalized'] = scaler.fit_transform(adata_gut.obs[[gp]])

color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=2,
                        ncols=2,
                        figsize=(15, 15))

# Ventral Gut
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[dorsalgut_cluster][0]}", # not normalized (to match original manuscript)
    color_map=color_map,
    spot_size=spot_size,
    title=f"Cthrc1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[dorsalgut_cluster][1]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Pdgfc GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

# Dorsal Gut
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[ventralgut_cluster][0]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Ihh GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[ventralgut_cluster][1]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Spint1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/gut_enriched_gps.svg",
            bbox_inches="tight")

### 2.5. All Emybros KNN 12

In [ ]:
model_label = "reference"
model_folder_path = f"{artifacts_folder_path}/{dataset}/models/{model_label}/{load_timestamps[4]}"
figure_folder_path = f"{artifacts_folder_path}/{dataset}/figures/{model_label}/{load_timestamps[4]}"


# Create required directories
os.makedirs(figure_folder_path, exist_ok=True)

model = NicheCompass.load(dir_path=model_folder_path,
                          adata=None,
                          adata_file_name=f"{dataset}_{model_label}.h5ad",
                          gp_names_key=gp_names_key)

In [ ]:
samples = model.adata.obs[sample_key].unique().tolist()

In [ ]:
# Identify and plot niches
latent_leiden_resolution = 0.35
latent_cluster_key = f"latent_leiden_{str(latent_leiden_resolution)}"

sc.tl.leiden(adata=model.adata,
             resolution=latent_leiden_resolution,
             key_added=latent_cluster_key,
             neighbors_key=latent_key)

sc.tl.leiden(adata=model.adata,
             resolution=0.05,
             key_added=latent_cluster_key,
             restrict_to=(latent_cluster_key, ["0"]),
             neighbors_key=latent_key)

latent_cluster_colors = create_new_color_dict(
    adata=model.adata,
    color_palette="default",
    cat_key=latent_cluster_key)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "latent_clusters_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=None,
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot CNS niches
forebrain_cluster = "3"
midbrain_cluster = "0,1"
hindbrain_cluster = "8"
floorplate_cluster = "15"

latent_cluster_colors[forebrain_cluster] = "#B497E7"
latent_cluster_colors[midbrain_cluster] = "#FF69B4"
latent_cluster_colors[hindbrain_cluster] = "#276A8C"
latent_cluster_colors[floorplate_cluster] = "#C38D9E"

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "forebrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[forebrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "midbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[midbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "hindbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[hindbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "floorplate_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[floorplate_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot gut niches
ventralgut_cluster = "2"
dorsalgut_cluster = "22"

latent_cluster_colors[ventralgut_cluster] = "#F6CF71"
latent_cluster_colors[dorsalgut_cluster] = "#9D88A2"

# Ventral Gut
save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "ventralgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[ventralgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "dorsalgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[dorsalgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Run differential GP testing
selected_cats = None
log_bayes_factor_thresh = 2.3

enriched_gps = model.run_differential_gp_tests(
    cat_key=latent_cluster_key,
    selected_cats=selected_cats,
    comparison_cats="rest",
    log_bayes_factor_thresh=log_bayes_factor_thresh)

# Check whether GPs are enriched
niche_gps = {}
niche_gps[forebrain_cluster] = ["PCSK1N_ligand_receptor_GP", "DKK1_ligand_receptor_GP"]
niche_gps[midbrain_cluster] = ["Fgf17_ligand_receptor_target_gene_GP", "Efna2_ligand_receptor_target_gene_GP"]
niche_gps[hindbrain_cluster] = ["Gdf10_ligand_receptor_target_gene_GP", "Fgf3_ligand_receptor_target_gene_GP"]
niche_gps[floorplate_cluster] = ["CALCA_ligand_receptor_GP", "Shh_ligand_receptor_target_gene_GP"]
niche_gps[ventralgut_cluster] = ["IHH_ligand_receptor_GP", "Spint1_ligand_receptor_target_gene_GP"]
niche_gps[dorsalgut_cluster] = ["Cthrc1_ligand_receptor_target_gene_GP", "PDGFC_ligand_receptor_GP"]

overlap_gps = {}
for niche, gps in niche_gps.items():
    niche_enriched_gps = model.adata.uns[differential_gp_test_results_key][
        model.adata.uns[differential_gp_test_results_key]["category"] == niche]["gene_program"].values.tolist()
    overlap_gps[niche] = [gp for gp in gps if gp in niche_enriched_gps]

# Correct sign of GPs
gp_summary_df = model.get_gp_summary()
model.add_active_gp_scores_to_obs()

for idx, row in gp_summary_df[gp_summary_df["gp_active"]].iterrows():
    if np.array(row["gp_source_genes_weights"]).sum() < 0:
        model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
    elif np.array(row["gp_source_genes_weights"]).sum() == 0:
        if np.array(row["gp_target_genes_weights"]).sum() < 0:
            model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
            
all_gps = []
for sublist in list(niche_gps.values()):
    all_gps.extend(sublist)

In [ ]:
# Plot CNS GPs
adata_top = model.adata[model.adata.obsm["spatial"][:,1] < 0]
adata_top = adata_top[adata_top.obs["sample"] == "embryo2"]

for gp in all_gps:
    if gp in adata_top.obs.columns:
        scaler = MinMaxScaler()
        # Fit and transform the column to normalize its values
        adata_top.obs[f'{gp}_normalized'] = scaler.fit_transform(adata_top.obs[[gp]])

color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=4,
                        ncols=2,
                        figsize=(15, 15))

# Forebrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[forebrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Pcsk1n GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[forebrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Dkk1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

# Midbrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[midbrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Fgf17 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[midbrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Efna2 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)

# Floor plate
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[floorplate_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Calca GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[floorplate_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Shh GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 1],
    show=False)

# Hindbrain
#sc.pl.spatial(
#    adata=adata_top,
#    color=f"{niche_gps[hindbrain_cluster][0]}_normalized",
#    color_map=color_map,
#    spot_size=spot_size,
#    title=f"Gdf10 GP",
#    legend_loc=None,
#    colorbar_loc="bottom",
#    ax=axs[3, 0],
#    show=False) # inactive GP
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[hindbrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Fgf3 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[3, 1],
    show=False)

plt.show()
fig.savefig(f"{figure_folder_path}/brain_enriched_gps_top.svg",
            bbox_inches="tight")

In [ ]:
# Plot gut GPs
adata_gut = model.adata[model.adata.obsm["spatial"][:,1] < 2.8]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,1] > -1.6]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,0] < 1.3]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,0] > -1.2]
adata_gut = adata_gut[adata_gut.obs["sample"] == "embryo2"]

for gp in all_gps:
    if gp in adata_gut.obs.columns:
        scaler = MinMaxScaler()
        # Fit and transform the column to normalize its values
        adata_gut.obs[f'{gp}_normalized'] = scaler.fit_transform(adata_gut.obs[[gp]])

color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=2,
                        ncols=2,
                        figsize=(15, 15))

# Ventral Gut
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[dorsalgut_cluster][0]}", # not normalized (to match original manuscript)
    color_map=color_map,
    spot_size=spot_size,
    title=f"Cthrc1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[dorsalgut_cluster][1]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Pdgfc GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

# Dorsal Gut
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[ventralgut_cluster][0]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Ihh GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[ventralgut_cluster][1]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Spint1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/gut_enriched_gps.svg",
            bbox_inches="tight")

### 2.6. All Emybros KNN 4

In [ ]:
model_label = "reference"
model_folder_path = f"{artifacts_folder_path}/{dataset}/models/{model_label}/{load_timestamps[5]}"
figure_folder_path = f"{artifacts_folder_path}/{dataset}/figures/{model_label}/{load_timestamps[5]}"


# Create required directories
os.makedirs(figure_folder_path, exist_ok=True)

model = NicheCompass.load(dir_path=model_folder_path,
                          adata=None,
                          adata_file_name=f"{dataset}_{model_label}.h5ad",
                          gp_names_key=gp_names_key)

In [ ]:
samples = model.adata.obs[sample_key].unique().tolist()

In [ ]:
# Identify and plot niches
latent_leiden_resolution = 0.35
latent_cluster_key = f"latent_leiden_{str(latent_leiden_resolution)}"

sc.tl.leiden(adata=model.adata,
             resolution=latent_leiden_resolution,
             key_added=latent_cluster_key,
             neighbors_key=latent_key)

latent_cluster_colors = create_new_color_dict(
    adata=model.adata,
    color_palette="default",
    cat_key=latent_cluster_key)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "latent_clusters_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=None,
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot CNS niches
forebrain_cluster = "6"
midbrain_cluster = "4"
hindbrain_cluster = "8"
floorplate_cluster = "16"

latent_cluster_colors[forebrain_cluster] = "#B497E7"
latent_cluster_colors[midbrain_cluster] = "#FF69B4"
latent_cluster_colors[hindbrain_cluster] = "#276A8C"
latent_cluster_colors[floorplate_cluster] = "#C38D9E"

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "forebrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[forebrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "midbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[midbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "hindbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[hindbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "floorplate_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[floorplate_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot gut niches
ventralgut_cluster = "5"
dorsalgut_cluster = "21"

latent_cluster_colors[ventralgut_cluster] = "#F6CF71"
latent_cluster_colors[dorsalgut_cluster] = "#9D88A2"

# Ventral Gut
save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "ventralgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[ventralgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "dorsalgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[dorsalgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Run differential GP testing
selected_cats = None
log_bayes_factor_thresh = 2.3

enriched_gps = model.run_differential_gp_tests(
    cat_key=latent_cluster_key,
    selected_cats=selected_cats,
    comparison_cats="rest",
    log_bayes_factor_thresh=log_bayes_factor_thresh)

# Check whether GPs are enriched
niche_gps = {}
niche_gps[forebrain_cluster] = ["PCSK1N_ligand_receptor_GP", "DKK1_ligand_receptor_GP"]
niche_gps[midbrain_cluster] = ["Fgf17_ligand_receptor_target_gene_GP", "Efna2_ligand_receptor_target_gene_GP"]
niche_gps[hindbrain_cluster] = ["Gdf10_ligand_receptor_target_gene_GP", "Fgf3_ligand_receptor_target_gene_GP"]
niche_gps[floorplate_cluster] = ["CALCA_ligand_receptor_GP", "Shh_ligand_receptor_target_gene_GP"]
niche_gps[ventralgut_cluster] = ["IHH_ligand_receptor_GP", "Spint1_ligand_receptor_target_gene_GP"]
niche_gps[dorsalgut_cluster] = ["Cthrc1_ligand_receptor_target_gene_GP", "PDGFC_ligand_receptor_GP"]

overlap_gps = {}
for niche, gps in niche_gps.items():
    niche_enriched_gps = model.adata.uns[differential_gp_test_results_key][
        model.adata.uns[differential_gp_test_results_key]["category"] == niche]["gene_program"].values.tolist()
    overlap_gps[niche] = [gp for gp in gps if gp in niche_enriched_gps]

# Correct sign of GPs
gp_summary_df = model.get_gp_summary()
model.add_active_gp_scores_to_obs()

for idx, row in gp_summary_df[gp_summary_df["gp_active"]].iterrows():
    if np.array(row["gp_source_genes_weights"]).sum() < 0:
        model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
    elif np.array(row["gp_source_genes_weights"]).sum() == 0:
        if np.array(row["gp_target_genes_weights"]).sum() < 0:
            model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
            
all_gps = []
for sublist in list(niche_gps.values()):
    all_gps.extend(sublist)

In [ ]:
# Plot CNS GPs
adata_top = model.adata[model.adata.obsm["spatial"][:,1] < 0]
adata_top = adata_top[adata_top.obs["sample"] == "embryo2"]

for gp in all_gps:
    if gp in adata_top.obs.columns:
        scaler = MinMaxScaler()
        # Fit and transform the column to normalize its values
        adata_top.obs[f'{gp}_normalized'] = scaler.fit_transform(adata_top.obs[[gp]])

color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=4,
                        ncols=2,
                        figsize=(15, 15))

# Forebrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[forebrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Pcsk1n GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[forebrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Dkk1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

# Midbrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[midbrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Fgf17 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[midbrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Efna2 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)

# Floor plate
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[floorplate_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Calca GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[floorplate_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Shh GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 1],
    show=False)

# Hindbrain
#sc.pl.spatial(
#    adata=adata_top,
#    color=f"{niche_gps[hindbrain_cluster][0]}_normalized",
#    color_map=color_map,
#    spot_size=spot_size,
#    title=f"Gdf10 GP",
#    legend_loc=None,
#    colorbar_loc="bottom",
#    ax=axs[3, 0],
#    show=False) # inactive GP
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[hindbrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Fgf3 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[3, 1],
    show=False)

plt.show()
fig.savefig(f"{figure_folder_path}/brain_enriched_gps_top.svg",
            bbox_inches="tight")

In [ ]:
# Plot gut GPs
adata_gut = model.adata[model.adata.obsm["spatial"][:,1] < 2.8]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,1] > -1.6]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,0] < 1.3]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,0] > -1.2]
adata_gut = adata_gut[adata_gut.obs["sample"] == "embryo2"]

for gp in all_gps:
    if gp in adata_gut.obs.columns:
        scaler = MinMaxScaler()
        # Fit and transform the column to normalize its values
        adata_gut.obs[f'{gp}_normalized'] = scaler.fit_transform(adata_gut.obs[[gp]])

color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=2,
                        ncols=2,
                        figsize=(15, 15))

# Ventral Gut
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[dorsalgut_cluster][0]}", # not normalized (to match original manuscript)
    color_map=color_map,
    spot_size=spot_size,
    title=f"Cthrc1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[dorsalgut_cluster][1]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Pdgfc GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

# Dorsal Gut
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[ventralgut_cluster][0]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Ihh GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[ventralgut_cluster][1]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Spint1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/gut_enriched_gps.svg",
            bbox_inches="tight")